In [1]:
""" An example showing how to save/restore models and retrieve weights. """
from __future__ import absolute_import, division, print_function
import tflearn
import tflearn.datasets.mnist as mnist

In [2]:
# MNIST Data
X, Y, testX, testY = mnist.load_data(one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


/home/ankdesh/installed/anaconda2/envs/tf0.11/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
/home/ankdesh/installed/anaconda2/envs/tf0.11/lib/python2.7/site-packages/tflearn/datasets/mnist.py:52: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [3]:
# Model
input_layer = tflearn.input_data(shape=[None, 784], name='input')
dense1 = tflearn.fully_connected(input_layer, 128, name='dense1')
dense2 = tflearn.fully_connected(dense1, 256, name='dense2')
softmax = tflearn.fully_connected(dense2, 10, activation='softmax')
regression = tflearn.regression(softmax, optimizer='adam',
                                learning_rate=0.001,
                                loss='categorical_crossentropy')

# Define classifier, with model checkpoint (autosave)
model = tflearn.DNN(regression, checkpoint_path='model.tfl.ckpt')

# Train model, with model checkpoint every epoch and every 200 training steps.
model.fit(X, Y, n_epoch=1,
          validation_set=(testX, testY),
          show_metric=True,
          snapshot_epoch=True, # Snapshot (save & evaluate) model every epoch.
          snapshot_step=500, # Snapshot (save & evalaute) model every 500 steps.
          run_id='model_and_weights')

Training Step: 860  | total loss: 0.28528
| Adam | epoch: 001 | loss: 0.28528 - acc: 0.9228 | val_loss: 0.30500 - val_acc: 0.9156 -- iter: 55000/55000
Training Step: 860  | total loss: 0.28528
| Adam | epoch: 001 | loss: 0.28528 - acc: 0.9228 | val_loss: 0.30500 - val_acc: 0.9156 -- iter: 55000/55000
--


In [4]:
# ------------------
# Retrieving weights
# ------------------

# Retrieve a layer weights, by layer name:
dense1_vars = tflearn.variables.get_layer_variables_by_name('dense1')
# Get a variable's value, using model `get_weights` method:
print("Dense1 layer weights:")
print(model.get_weights(dense1_vars[0]))
# Or using generic tflearn function:
print("Dense1 layer biases:")
with model.session.as_default():
    print(tflearn.variables.get_value(dense1_vars[1]))

Dense1 layer weights:
[[ 0.02016847 -0.0120432  -0.00146774 ..., -0.01501851 -0.00607988
  -0.0128871 ]
 [ 0.00848555  0.00670409 -0.00505784 ..., -0.01526591 -0.00701048
  -0.00565379]
 [-0.00534527 -0.0087175  -0.01811047 ..., -0.00551683  0.03437988
  -0.01978657]
 ..., 
 [-0.02248678  0.02778613  0.00687615 ..., -0.01517815  0.01511721
  -0.01216448]
 [-0.01166681 -0.03815222  0.0195404  ...,  0.020554   -0.00860342
   0.00645357]
 [-0.00197658 -0.01563451 -0.0167937  ..., -0.00785239  0.02784383
  -0.00275727]]
Dense1 layer biases:
[-0.00483045 -0.02626631 -0.08924185 -0.0346124   0.02417278 -0.01819696
 -0.04836737 -0.03162532  0.026753    0.01777712  0.0115912  -0.04100706
  0.0054957  -0.00875653  0.04345272  0.03819585 -0.01714033 -0.13413471
  0.01778085  0.03894889 -0.00782998  0.00480117 -0.07080938 -0.02073469
  0.08067903 -0.04100002 -0.02849869  0.01947469  0.08119599  0.02877793
  0.09796295  0.04716418 -0.00105663 -0.01588985 -0.02753695  0.02431668
  0.04482774 -0.026

In [5]:
# It is also possible to retrieve a layer weights through its attributes `W`
# and `b` (if available).
# Get variable's value, using model `get_weights` method:
print("Dense2 layer weights:")
print(model.get_weights(dense2.W))
# Or using generic tflearn function:
print("Dense2 layer biases:")
with model.session.as_default():
    print(tflearn.variables.get_value(dense2.b))

Dense2 layer weights:
[[ 0.01055639  0.08620375  0.04862928 ..., -0.01440148  0.00888839
   0.02866616]
 [ 0.0339524  -0.00818544 -0.00485797 ..., -0.01178635 -0.01184989
   0.02456975]
 [-0.00547604  0.02584333  0.04844898 ...,  0.06854951 -0.00530339
   0.05393078]
 ..., 
 [-0.01961643 -0.07964525 -0.04223059 ..., -0.03262559 -0.03759835
  -0.03962386]
 [-0.0515429  -0.02473955 -0.03541492 ...,  0.00554273  0.01656909
  -0.04296798]
 [ 0.08400296  0.08940421  0.06693238 ...,  0.00980482 -0.00180459
   0.03236808]]
Dense2 layer biases:
[ 0.00044376 -0.04596734 -0.02944141  0.01140089 -0.08871038  0.00932581
  0.07159516  0.01199317 -0.00751944 -0.01700861  0.02900777 -0.02915867
  0.03783266  0.01565566 -0.03015781  0.01471736  0.00040163  0.06029579
  0.04535449 -0.01199872  0.04518174  0.05232766  0.01893858 -0.01036596
  0.0747833  -0.01804553 -0.00250435 -0.03463389  0.05454005  0.08295877
 -0.08360121  0.08016646 -0.01946675  0.09564856 -0.00541074 -0.00456367
 -0.06820051 -0.022